<a href="https://colab.research.google.com/github/NicoCaz/Resumen_video_yt/blob/main/Resumen_de_video_de_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install pytube
!sudo apt-get remove -y youtube-dl
!sudo apt update && sudo apt install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2cfq97ln
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2cfq97ln
  Resolved https://github.com/openai/whisper.git to commit ad3250a846fe7553a25064a2dc593e492dadf040
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.5 MB/s eta 0:00:00
  Created wheel for open

Parte del codigo donde se descarga el video de youtube

In [3]:
import os
from pytube import YouTube

#Cambia el link que quieras
LINK = "https://www.youtube.com/watch?v=4bxpsvcW2mc&t=480s&ab_channel=ElizabethFilips"

# Obtener objeto YouTube
yt = YouTube(LINK)

# Obtener objeto Stream de audio
audio_stream = yt.streams.filter(only_audio=True).first()

# Descargar archivo de audio en una ubicación temporal
temp_file_path = audio_stream.download()

# Renombrar archivo a "audio.mp3" y mover a la carpeta actual
os.rename(temp_file_path, os.path.join(os.getcwd(), "audio.mp3"))



In [4]:
import whisper
model = whisper.load_model("base")
result = model.transcribe("audio.mp3")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 58.6MiB/s]
/usr/local/lib/python3.9/dist-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [5]:
nombre_archivo = "texto_original.txt"

with open(nombre_archivo, "w") as archivo:
    archivo.write(result["text"])

# Cerrar el archivo
archivo.close()

Parte del codigo donde se genera el resumen

In [6]:
!pip install beautifulsoup4
!pip install urllib3
!pip install nltk
!python -m nltk.downloader punkt
!python -m nltk.downloader stopwords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
import bs4 as bs  
import urllib.request  
import re
import nltk
from nltk import word_tokenize,sent_tokenize
import heapq  


In [8]:

# Tamaño del resumen, cuando mas grande el numero mas extenso el resumen
tamanio_resumen = 150

# Función para formatear el artículo
def formatear_articulo(texto):
    texto_formateado = re.sub('[^a-zA-Z]', ' ', texto)
    texto_formateado = re.sub(r'\s+', ' ', texto_formateado)
    return texto_formateado

# Función para calcular la frecuencia de palabras
def calcular_frecuencia_palabras(texto_formateado):
    palabras = nltk.word_tokenize(texto_formateado.lower())
    stopwords = nltk.corpus.stopwords.words('spanish')
    frecuencia_palabras = {word: palabras.count(word) for word in palabras if word not in stopwords}
    return frecuencia_palabras

# Función para obtener las oraciones
def obtener_oraciones(texto):
    return nltk.sent_tokenize(texto)

# Función para calcular la frecuencia de las oraciones
def calcular_frecuencia_oraciones(oraciones, frecuencia_palabras):
    max_oracion = {}
    for sent in oraciones:
        for word in nltk.word_tokenize(sent.lower()):
            if word in frecuencia_palabras.keys():
                if len(sent.split(' ')) < tamanio_resumen:
                    if sent not in max_oracion.keys():
                        max_oracion[sent] = frecuencia_palabras[word]
                    else:
                        max_oracion[sent] += frecuencia_palabras[word]
    return max_oracion

# Función para generar el resumen
def generar_resumen(texto):
    texto_formateado = formatear_articulo(texto)
    frecuencia_palabras = calcular_frecuencia_palabras(texto_formateado)
    oraciones = obtener_oraciones(texto)
    max_oracion = calcular_frecuencia_oraciones(oraciones, frecuencia_palabras)
    resumen_oracion = heapq.nlargest(7, max_oracion, key=max_oracion.get)
    resumen = ' '.join(resumen_oracion)
    return resumen


resumen = generar_resumen(result["text"])
print(resumen)


nombre_archivo = "salida.txt"

with open(nombre_archivo, "w") as archivo:
    archivo.write(resumen)

# Cerrar el archivo
archivo.close()





Then I'll have art, so sometimes if I see like getting to know yourself, if there's some relevant art that I find or nine, which is quite inspiring, I will try to get a resource, I don't have a resource for this, but and I will put it there because if I need to talk about it, I might link the art to it too. I honestly find this so, so useful because it means that every single book that I read, I analyse so much and I spent a lot of time kind of connecting to previous things that I know and previous things that I found and really thinking, is this relevant to my life? So every time I create a note, I will go all the way to my second brain and look through every single one on my list and think, oh, this is slightly relevant, so this could be a secondary source on something, it could be a tertiary source on something, it could be a primary source on something else, and this is how I keep populating my second brain with very interesting connections to me that were not there previously. To 

In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# cargar modelo y tokenizador
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# definir texto de entrada
input_text = result["text"]
max_input_length = model.config.max_position_embeddings - 2 # tomar en cuenta los tokens especiales [CLS] y [SEP]
if len(input_text) > max_input_length:
    # segmentar el texto
    input_segments = [input_text[i:i+max_input_length] for i in range(0, len(input_text), max_input_length)]
else:
    input_segments = [input_text]

# definir preguntas
questions = [
   "¿De qué se está hablando en el texto?"
]

# hacer inferencia y mostrar resultados
for q in questions:
    response_segments = []
    for segment in input_segments:
        input_sequence = f"{segment.strip()} {tokenizer.sep_token} {q}"
        input_ids = tokenizer.encode(input_sequence, add_special_tokens=True)
        max_length = model.config.max_position_embeddings - input_ids.size(1) # establecer la longitud máxima en función del tamaño máximo de posición del modelo
        attention_mask = torch.ones(input_ids.shape, dtype=torch.long) # crear la máscara de atención
        outputs = model.generate(input_ids, max_length=max_length, do_sample=True, pad_token_id=model.config.eos_token_id, 
                                 attention_mask=attention_mask) # generar la respuesta
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response_segments.append(response)

    # concatenar las respuestas de los segmentos
    response = " ".join(response_segments)

    print(f"Pregunta: {q}")
    print(f"Respuesta: {response}\n")
